In [81]:
import numpy as np
import tqix
import sys
sys.path.insert(1, '../')
import qiskit, qtm.encoding, qtm.base


def create_basic_vector(num_qubits: int):
    """Generate list of basic vectors

    Args:
        num_qubits (int): number of qubits

    Returns:
        np.ndarray: |00...0>, |00...1>, ..., |11...1>
    """
    bs = []
    for i in range(0, 2**num_qubits):
        b = np.zeros((2**num_qubits, 1))
        b[i] = 1
        bs.append(b)
    return bs

def calculate_sigma(U: np.ndarray, b: np.ndarray):
    """Calculate measurement values

    Args:
        U (np.ndarray): operator
        b (np.ndarray): basic vector

    Returns:
        np.ndarray: sigma operator
    """
    return (np.conjugate(np.transpose(U)) @ b @ np.conjugate(np.transpose(b)) @ U)


Step 1. Create $\rho_{unk}$. It's psi

In [82]:
num_qubits = 2
num_observers = 10
psi = 2*np.random.rand(2**num_qubits)-1
psi = psi / np.linalg.norm(psi)
rho = qiskit.quantum_info.DensityMatrix(psi).data
qc = qiskit.QuantumCircuit(num_qubits, num_qubits)
qc.initialize(psi)

Step 2. Create Us = $U_1, U_2, ..., U_{num\_observer}$ and bs = $|0\rangle, |1\rangle, ..., |2^n-1\rangle$.

bs[0] = [1, 0, ... 0], bs[$2^n-1$] = [0, 0, ..., 1]

In [83]:
Us, bs = [], []
for i in range(0, num_observers):
    random_psi = 2*np.random.rand(2**num_qubits)-1
    random_psi = random_psi / np.linalg.norm(random_psi)
    encoder = qtm.encoding.Encoding(random_psi, 'amplitude_encoding')
    qcs = (qc.copy()).compose(encoder.qcircuit)
    U = (qiskit.quantum_info.Operator(encoder.qcircuit).data)
    Us.append(U)
    bs = create_basic_vector(num_qubits)


Step 3. Calculate $\sigma_i=\sigma_i^{(0)}, \sigma_i^{(1)}, ..., \sigma_i^{(2^n-1)}$
with $\sigma_i^{(j)}=U_i^{\dagger}|j\rangle\langle j|U_i$

In [84]:
sigmass = []
for i in range(0, num_observers):
    sigmas = []
    for b in bs:
        sigma = calculate_sigma(Us[i], b)
        sigmas.append(sigma)
    sigmass.append(sigmas)

Step 4: Calculate $\mu(\rho_{unk})=\frac{1}{num\_ observer}\sum_{i=1}^{num\_observer}\sum_{b=0}^{2^n-1} \text{Tr}(\sigma_i^{(b)}\rho_{unk})\sigma_i^{(b)}$

In [85]:
M = np.zeros((2**num_qubits, 2**num_qubits), dtype=np.complex128)
for i in range(0, num_observers):
    for j in range(0, 2**num_qubits):
        k = sigmass[i][j]
        M += np.trace(k @ rho)*k
M /= num_observers


Step 5: Calculate $\tilde{\rho}=\frac{1}{num\_ observer}\sum_{i=1}^{num\_observer}(\sum_{b=0}^{2^n-1} (\text{Tr}(\sigma_i^{(b)}).\mu^{-1}(\rho_{unk}).\sigma_i^{(b)}))$

In [86]:
rho_hat = np.zeros((2**num_qubits, 2**num_qubits), dtype=np.complex128)
for i in range(0, num_observers):
    for j in range(0, 2**num_qubits):
        k = sigmass[i][j]
        rho_hat += np.trace(k)*np.linalg.inv(M) * k
rho_hat /= num_observers


In [89]:
print("p", rho)
print("p~", rho_hat)
fidelity = qtm.base.trace_fidelity(rho, rho_hat)
trace = qtm.base.trace_distance(rho, rho_hat)
print("Fidelity: ", fidelity)
print("Trace: ", trace)

p [[ 3.58356322e-01+0.j -2.78685361e-01-0.j  1.12114364e-02+0.j
   3.90058767e-01+0.j]
 [-2.78685361e-01+0.j  2.16727112e-01+0.j -8.71887288e-03+0.j
  -3.03339614e-01+0.j]
 [ 1.12114364e-02+0.j -8.71887288e-03-0.j  3.50757890e-04+0.j
   1.22032703e-02+0.j]
 [ 3.90058767e-01+0.j -3.03339614e-01-0.j  1.22032703e-02+0.j
   4.24565809e-01+0.j]]
p~ [[ 4.98855101e+00+0.j  0.00000000e+00+0.j -6.93889390e-19+0.j
  -6.93889390e-19+0.j]
 [ 0.00000000e+00+0.j  6.20841056e+00+0.j  0.00000000e+00+0.j
   9.99200722e-17+0.j]
 [ 4.16333634e-18+0.j  0.00000000e+00+0.j  6.55014286e+00+0.j
   0.00000000e+00+0.j]
 [-1.38777878e-18+0.j  9.99200722e-17+0.j  0.00000000e+00+0.j
   4.67586275e+00+0.j]]
Fidelity:  (2.2593711320496475+8.724866197282509e-09j)
Trace:  10.711483596627007


<img width = '600px' src = '../../images/shadow_tomography1.jpg'>
<img width = '600px' src = '../../images/shadow_tomography2.jpg'>